# CAI Lab Session 7: Implementing Pagerank

In this session you will:

- implement an efficient version of Pagerank on directed graphs
- apply it to couple of examples, including a graph defined by airports and flights

## 1. Pagerank

In class we have explained the iterative method for computing PageRank using matricial notation (i.e. the ``power method''). 
The iterative step was the matrix-times-vector product $P(t) = M^T*P(t-1)$,
but then we replaced $M$ with the Google matrix $G = d*M + (1-d)/n * J$, where $d$ is the \emph{damping factor}. 
This was done in order to guarantee a unique solution and its fast convergence to it.

In this lab, we assume that a graph $G=(V,E)$ is given to us as a table of vertices + for each one, a list
of successors. This is known as the _adjacency list_ representation. In cases where we have significantly fewer edges than the maximum possible (e.g. in a graph with $n$ nodes we may have $O(n)$ edges instead of the maximum $O(n^2)$ possible. In this case, the adjacency list representation is much more compact than the matrix. Additionally for very large graphs we may not even be able to fit the full matrix into main memory, and would need instead to process few adjacency lists at a time.

An efficient implementation of the formula using an adjacency list representation of $G$
instead of a matrix $M$ would be:

```
 1. n = number of vertices in G
 2. P = any vector of length n and sum 1 (for example, the all 1/n vector)
 3. d = the chosen damping factor, between 0 and 1
 4. while not stopping condition:
 5.    for i in V:
 6.      L = [ j for (j,i) in E ]
 7.      Pnew(i) = (1-d)/n
 8.      for j in L:
 9:          Pnew(i) += d * P(j)/out(j)
10:    compute a distance between Pnew and P (to be used in stopping condition)
11:    P = Pnew
```

__Silly question:__ How do we know that the division by `out(j)` in line 9 does not give an error?

The stopping condition we propose is that `P` and `Pnew` are sufficiently close
under some distance you need to compute. "Close" is up to you: 0.01, 0.00001, 0.00000001, etc.

The damping factor $d$ is of your choice. Popular ones are between 0.8 and 0.9. Something 
you can investigate is how different choices affect the solution and how they affect the computation time.

## 2. Pagerank without inverting the graph

The scheme above is all fine for graphs that fit in RAM.
But this need not be the case. It is not uncommon to have graphs so large that
at least the lists of edges have to be in disk. And, most likely, 
we will have a list of _outgoing_ edges, not _incoming_ edges.

In other words, the algorithm above uses the list `[ j for (j,i) in E ]`. 
But if we are given $G$ and not its inversion, what we will have instead is `[ j for (i,j) in E ]`.

Given this, we have two options:


1. _Invert the graph._ This is similar to the problem of computing the inverted index.
As it has very low disk locality if implemented naively, this takes time.

2. _Change the algorithm_ so that we compute pageranks incrementally. This is similar to 
what we did for computing tf-idf: invert the loops, and keep a set of partially computed
tf-idfs (here, of partially computed pageranks). Similar to this:

```
 1. n = number of vertices in G
 2. P = any vector of length n and sum 1 (for example, the all 1/n vector)
 3. d = the chosen damping factor, between 0 and 1
 4. while not stopping condition:
 5.    Pnew = the all (1-d)/n vector
 6.    for i in V :
 7.        L = [ j for (i,j) in E ]. // forward adjacency list for node i
 8.        for j in L:
 9:            Pnew(j) += d * P(i)/out(i)
10:    compute a distance between Pnew and P
11:    P = Pnew
```


## 3. The input files

The following two files have been downloaded from [Open Flights](http://openflights.org/data.html).


- `airports.txt` contains a list of airports from the world. The first fields are: an OpenFlights airport identifier, name of the airport, main city it serves, country, 3 letter IATA code, 4 letter ICAO code, and other stuff. (Only major airports have IATA codes). As an example, the first two lines of this file are as follows:


In [4]:
!head -n 2 airports.txt

1,"Goroka","Goroka","Papua New Guinea","GKA","AYGA",-6.081689,145.391881,5282,10,"U"
2,"Madang","Madang","Papua New Guinea","MAG","AYMD",-5.207083,145.7887,20,10,"U"



- `routes.txt` contains a list of routes from the world. The first fields are: an airline code, an OpenFlights airline code, an origin airport code (3 letter IATA code or 4 letter ICAO code), same with OpenFlight code, a destination airport code (3 letter IATA code or 4 letter ICAO code), then other stuff.



In [5]:
!head -n 2 routes.txt

2B,410,AER,2965,ASF,2966,,0,CR2
2B,410,AER,2965,GOJ,4274,,0,CR2



Note that there is no guarantee that each airport mentioned in `routes.txt` has an entry in `airports.txt`. 
There may also be dangling nodes, that is, airports with some incoming route and no outgoing routes. 
__As you know, Pagerank needs to be patched to deal with these types of nodes.__

We provide reading functions for these two files to return a dictionaries representing routes and airports.
The way these functions deal with dangling nodes is by simply removing them.

In [2]:
def read_airports():
# sample line:
# 1382,"Charles De Gaulle","Paris","France","CDG","LFPG",49.012779,2.55,392,1,"E"
    airportsTxt = open("airports.txt", "r", encoding="utf8");
    cont = 0
    airport_dict = {}
    for line in airportsTxt.readlines():
        try:
            temp = line.split(',')
            airp_name = temp[4][1:-1]
            airport_dict[airp_name] = (
                airp_name+
                " ("+temp[1][1:-1]+", "+
                temp[2][1:-1]+", "+
                temp[3][1:-1]+")"
                )
        except Exception as inst:
            # print("incorrect line in airports:",line)
            pass
    airportsTxt.close()
# for the sample line, it adds to dict the pair
# "CDG": "CDG (Charles de Gaulle, Paris, France)"
    print(airport_dict["CDG"])
    print(len(airport_dict), "airports read successfully")
    return airport_dict

def read_routes(airp):
# sample line:
# AB,214,CDG,1382,VIE,1613,Y,0,320 321
# note: there are no " quotes around airport names, unlike in airports.txt
    routesTxt = open("routes.txt", "r", encoding="utf8");
    route_dict = dict()
    nroutes = 0
    for line in routesTxt.readlines():
        try:
            temp = line.split(',')
            origin_id = temp[2]
            dest_id = temp[4]
            origin_airp = airp[origin_id]
            dest_airp = airp[dest_id]
            if origin_id not in route_dict:
                route_dict[origin_id] = [] # empty LIST
            route_dict[origin_id].append(dest_id)
            nroutes += 1
        except Exception as inst:
            # print("incorrect line, or unknown airport, in routes:", line)
            pass
    routesTxt.close()
    print(nroutes, "routes read successfully")

    # remove dangling nodes
    filtered = dict()
    for i in route_dict:
        all_destinations = [ j for j in route_dict[i] if j in route_dict]
        if len(all_destinations):
            filtered[i] = all_destinations

    return filtered

In [3]:
airports_dict = read_airports()
airports_dict["CDG"]

CDG (Charles De Gaulle, Paris, France)
5742 airports read successfully


'CDG (Charles De Gaulle, Paris, France)'

In [9]:
routes = read_routes(airports_dict)
routes["CDG"]

68292 routes read successfully


['ATH',
 'HER',
 'SKG',
 'TBS',
 'BOS',
 'DFW',
 'JFK',
 'KUL',
 'LHR',
 'MIA',
 'ORD',
 'VIE',
 'YUL',
 'YYZ',
 'ABJ',
 'ABV',
 'ABZ',
 'AGP',
 'ALG',
 'AMM',
 'AMS',
 'ARN',
 'ATH',
 'ATL',
 'AUH',
 'BCN',
 'BEG',
 'BES',
 'BEY',
 'BHX',
 'BIO',
 'BKK',
 'BKO',
 'BLL',
 'BLQ',
 'BLR',
 'BOD',
 'BOG',
 'BOM',
 'BOS',
 'BRE',
 'BRS',
 'BSL',
 'BUD',
 'BZV',
 'CAI',
 'CAN',
 'CCS',
 'CFE',
 'CGN',
 'CKY',
 'CLY',
 'CMN',
 'COO',
 'CPH',
 'CVG',
 'CWL',
 'DEL',
 'DKR',
 'DLA',
 'DTW',
 'DUB',
 'DUS',
 'DXB',
 'EDI',
 'EMA',
 'EVN',
 'EWR',
 'EXT',
 'EZE',
 'FCO',
 'FIH',
 'FLR',
 'FRA',
 'FSC',
 'GIG',
 'GOA',
 'GOT',
 'GRU',
 'GVA',
 'GYD',
 'HAJ',
 'HAM',
 'HAN',
 'HAV',
 'HEL',
 'HKG',
 'HND',
 'IAD',
 'IAH',
 'ICN',
 'IST',
 'JED',
 'JFK',
 'JNB',
 'KBP',
 'KIX',
 'KUL',
 'LAD',
 'LAX',
 'LBV',
 'LCA',
 'LED',
 'LFW',
 'LHR',
 'LIM',
 'LIN',
 'LIS',
 'LJU',
 'LOS',
 'LUX',
 'LYS',
 'MAD',
 'MAN',
 'MEX',
 'MIA',
 'MPL',
 'MRS',
 'MRU',
 'MSP',
 'MUC',
 'NAP',
 'NBO',
 'NCE',
 'NCL',


## 4. To do

1.  Make sure you understand the first version of Pagerank above. Really, make sure, do not just transcribe it. 
Make sure you see the connection with the matrix formulation in the course slides. 

2.  Now go to the 2nd version (disk friendly) of Pagerank. Make sure you understand
the key difference, and why it should be better with data in disk. 

3.  Implement it (the 2nd version) and try it on the following simple graph (graph used in course notes). Check that the pagerank values you obtain are those stated in the course notes from which the example comes from. Test the effect of changing the damping factor. If they don't come out right, first thing to check is that after each iteration
the sum of `P` is 1 (or close to 1 except for tiny rounding errors). If it's not, you are doing something wrong in the iteration step.

In [4]:
import numpy as np
import pprint

In [33]:
# a simple 4-node graph from the course slides
simple_graph = {
  1: [1, 3, 4],
  2: [1, 4],
  3: [2, 4],
  4: [2]
}

def compute_pageranks(g, d):
    """ 
    this function takes a dictionary g representing a graph and damping factor d, 
    and outputs a list of (vertexname, pagerank) values for each vertex
    """
    n = len(g)
    P = np.array([1/n]*n, dtype=np.float32)
    d = 1
    dist = np.inf
    
    while dist > 0.0001: 
        Pnew = np.array([(1-d)/n]*n, dtype=np.float32) 
        for i in g:
            for j in g[i]:
                Pnew[j-1] += d * P[i-1]/len(g[i])
        dist = np.linalg.norm(P-Pnew)
        P = Pnew
        
    return list(zip(g.keys(), P))

In [34]:
compute_pageranks(simple_graph, 0.85)

[(1, 0.26088116), (2, 0.347804), (3, 0.08695028), (4, 0.30436447)]

4.  Now apply the pagerank algorithm to the graph of airport routes and look at the result. Are you surprised by the airports at the top (with most pagerank)?

5. Experiment with different values of the damping factor, and how it affects the convergence rate (iterations and time).

In [29]:
def compute_pageranks(g, d):
    """ 
    This function takes a dictionary g representing a graph and damping factor d, 
    and outputs a list of (vertexname, pagerank) values for each vertex
    """
    n = len(airports_dict)
    vertex_names = list(airports_dict.keys())
    vertex_indices = {vertex_names[i]: i for i in range(n)}

    P = np.array([1/n] * n, dtype=np.float32)
    dist = np.inf
    
    while dist > 0.01:
        Pnew = np.array([(1-d)/n] * n, dtype=np.float32) 
        for vertex in vertex_names: 
            if vertex in g: 
                for neighbor in g[vertex]:
                    Pnew[vertex_indices[neighbor]] += d * P[vertex_indices[vertex]] / len(g[vertex])
            else:  
                Pnew += d * P[vertex_indices[vertex]]/n
                    
        dist = np.linalg.norm(P - Pnew)
        P = Pnew
        
    return list(zip(vertex_names, P)), sum(P)

In [30]:
page_ranking, sum_of_pageranks = compute_pageranks(routes, 0.85)

In [31]:
sorted_page_ranking = sorted(page_ranking, key=lambda x: x[1], reverse=True)
sorted_page_ranking

[('DEN', 0.004749385),
 ('LAX', 0.0042391727),
 ('ORD', 0.0041284547),
 ('SYD', 0.0038502535),
 ('DME', 0.0037037404),
 ('ATL', 0.0034876927),
 ('DFW', 0.0033604421),
 ('BOG', 0.0033365716),
 ('SIN', 0.0032740852),
 ('PEK', 0.0031487786),
 ('LHR', 0.0030626848),
 ('CDG', 0.0029781386),
 ('FRA', 0.0029466164),
 ('MEX', 0.0028325622),
 ('IST', 0.0027802119),
 ('BNE', 0.002753704),
 ('YYZ', 0.0027245276),
 ('DXB', 0.0027216966),
 ('AMS', 0.0026973186),
 ('AEP', 0.0026750367),
 ('JNB', 0.002627411),
 ('MSP', 0.0025944975),
 ('ICN', 0.0025524755),
 ('JFK', 0.0025456122),
 ('MNL', 0.002504109),
 ('DEL', 0.0024893952),
 ('BKK', 0.0024451732),
 ('LED', 0.0024026064),
 ('HND', 0.0023183438),
 ('YVR', 0.0023172065),
 ('ANC', 0.0022945257),
 ('KUL', 0.0022801443),
 ('PVG', 0.0022711877),
 ('MIA', 0.0022705055),
 ('IAH', 0.0022641905),
 ('BCN', 0.0022608226),
 ('HKG', 0.0022606165),
 ('CLT', 0.0022506055),
 ('SEA', 0.002247181),
 ('AKL', 0.002160883),
 ('BOM', 0.002057973),
 ('SFO', 0.0020411296),